In [1]:
import math
import numpy as np

class QFromat:
    
    def __init__(self, m: int, n: int, U: bool=False):
        '''
        :param m: Integer bits 
        :param n: Fractional bits
        :param U: Signed?
        '''
        self.m = m
        self.n = n
        self.size = m + n 
        self.signed = U
        
        if(self.signed):
            self.minimum = 0
            self.maximum = pow(2, self.m) - pow(2, -self.n)
            self.resolution = pow(2, -self.n)
            self.wordSize = pow(2, self.size)
            self.decRange = [self.minimum, self.maximum]
            self.hexRange = [hex(0 - round(self.minimum / self.resolution)), hex(0 + round(self.maximum / self.resolution))]
        else:
            self.minimum = -pow(2, self.m - 1)
            self.maximum = pow(2, self.m - 1) - pow(2, -self.n)
            self.resolution = pow(2, -self.n)
            self.wordSize = pow(2, self.size)
            self.decRange = [self.minimum, self.maximum]
            self.hexRange = [hex(0 - round(self.minimum / self.resolution)), hex(0 + round(self.maximum / self.resolution))]    
        
        self.headroom = (self.maximum - self.minimum + self.resolution)
            
    def RtoF(self, input_data: hex) -> float:
        if(self.wordSize > input_data > self.maximum / self.resolution):
            return input_data * self.resolution - self.wordSize * self.resolution
        elif(self.maximum / self.resolution >= input_data >= 0):
            return input_data * self.resolution
        else:
            # overflow condition
            pass
    
    def FtoR(self, input_data: float) -> hex:
        if(self.maximum >= input_data >= 0):        
            input_data = (input_data / self.resolution)
            return hex(round(input_data))
        elif(self.minimum <= input_data < 0):
            input_data = (input_data / self.resolution)
            return hex(round(input_data) + self.wordSize)
        else:
            # overflow condition
            pass
        
    def Error(self, input_data: float) -> None:
        # Representation Error
        integer = self.FtoR(input_data)
        representation = self.RtoF(int(integer, 16))
        error = abs(input_data - representation)
        print("Input Value         : ", input_data)
        print("Integer Value       : ", integer)
        print("Fixed-point Value   : ", representation)
        print("Representation Error: ", error)
        print("Headroom            : ", self.headroom)
        
        if(abs(error) == 0):
            print("dBFS                : ", 0)
        else:
            print("dBFS                : ", round(20 * math.log(error/self.headroom, 10), 2))


In [2]:
# print(obj.FtoR(-2.5))
# print(obj.FtoR(-2))
# print(obj.FtoR(-1.5))
# print(obj.FtoR(-1))
# print(obj.FtoR(-0.5))
# print(obj.FtoR(0))
# print(obj.FtoR(0.5))
# print(obj.FtoR(1))
# print(obj.FtoR(1.5))
# print(obj.FtoR(2.0))

In [3]:
# for i in range(64):
#     print(i, hex(i), obj.RtoF(i)) 

Gain recovery constan. Controls how fast or slow the gain change should recover; a high value implies slower recovery: 
    
Q0.15, 
0 “ 0x0000, 
0.5 “ 0x4000, 
1 “ 0x7FFF, 

Gc(hex) “ dec2hex(round(Gc * 2^15))

Maximum window size in ms, not used when delay is set to a nonzero value: 

Q0.15, 
8ms “ 0x62, 
MaxWait(hex) “ dec2hex(round(MaxWait(ms)*2^15))

Threshold in dB to limit peaks: 

Q2.13, 
0 dB “ 0x2000, 
Thresh(hex) “ dec2hex(round(4096*10^ [Thresh(dB)/20]))

Makeup gain in dB to increase overall signal level after peak limiting: 

Q7.8, 
0 dB “ 0x0100, 
Gain(hex) “ dec2hex(round(256*10^ [Gain(dB)/20]))

Static input gain when aigMode is set to 0: 

Q3.12, 0 dB = 0x1000, 

staticGainL16Q12 = dec2hex(round(10^(staticGaindB /20) * 4096))

Attack/release time for AIG update: 

Q1.31, 
5000 ms = 0x0001CD5C at 8 kHz, 
10000 ms = 0x0000E6AF at 8kHz, 
gainAtRtUL32Q31 = dec2hex(round((1-exp(-2200 /fs/gainAtRtms))*2^31))